In [1]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/hjkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hjkim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hjkim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df = pd.read_csv('../entity_extraction/Preprocessed_Data/H01L_2020-2022_9585_df.csv')
df

,index,publication_title,publication_number,publication_date,application_type,sections,section_classes,section_class_subclasses,section_class_subclass_groups,abstract,descriptions,claims,year,background
0,0,Wafer polishing system,US10525568,20200107,utility,"['B', 'H']","['B24', 'H01']","['B24B', 'H01L']","['B24B 37/04', 'B24B 37/34', 'B24B 57/02', 'H0...",The wafer polishing system is disclosed. The w...,CROSS REFERENCE TO RELATED APPLICATION\nThis a...,1. A wafer polishing system comprising:\na pol...,H01L,"Recently, the high integration of semiconducto..."
1,1,Slurry composition for CMP and polishing metho...,US10526508,20200107,utility,"['C', 'H']","['C09', 'H01']","['C09G', 'H01L']","['C09G 1/02', 'H01L 21/304', 'H01L 21/321', 'H...",Provided are a slurry composition for CMP and ...,TECHNICAL FIELD\nThe present invention relates...,1. A slurry composition for chemical-mechanica...,H01L,With an increase in the degree of integration ...
2,2,Methods for controlling the substrate temperat...,US10526705,20200107,utility,"['C', 'H']","['C23', 'H01']","['C23C', 'H01L']","['C23C 16/46', 'C23C 16/458', 'H01L 21/687', '...","In a CVD reactor, flushing gases of different ...",RELATED APPLICATIONS\nThis application is a Di...,1. A method for depositing a plurality of laye...,H01L,Such a CVD reactor is described in JP 2002-146...
3,3,In-situ metrology method for thickness measure...,US10527407,20200107,utility,"['G', 'H', 'C']","['G01', 'H01', 'C23']","['G01B', 'H01L', 'C23C', 'H01J']","['G01B 11/06', 'H01L 21/66', 'G01B 7/06', 'C23...",Embodiments of the present disclosure relate t...,CLAIM OF PRIORITY UNDER 35 U.S.C. 119\nThis ap...,"1. A processing chamber, comprising:\na substr...",H01L,Field\nEmbodiments of the present disclosure r...
4,4,Semiconductor inspection device,US10527648,20200107,utility,"['G', 'H']","['G01', 'H01']","['G01R', 'H01L']","['G01R 1/073', 'G01R 1/067', 'H01L 21/67']","According to the present invention, a semicond...",BACKGROUND OF THE INVENTION\nField\nThe presen...,1. A semiconductor inspection device comprisin...,H01L,Field\nThe present invention relates to a semi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9580,2571,Method for manufacturing semiconductor device,US11538921,20221227,utility,['H'],['H01'],['H01L'],"['H01L 21/336', 'H01L 29/40', 'H01L 29/66', 'H...","A source electrode (5), a drain electrode (6) ...",FIELD\nThe present invention relates to a meth...,1. A method for manufacturing a semiconductor ...,H01L,A gate electrode having a T-shaped cross secti...
9581,2572,Method for etching back hard mask layer on top...,US11538923,20221227,utility,['H'],['H01'],['H01L'],"['H01L 29/66', 'H01L 21/02', 'H01L 21/033', 'H...",A method for etching back a hard mask layer on...,CROSS-REFERENCES TO RELATED APPLICATIONS\nThis...,1. A method for semiconductor process for maki...,H01L,With continuous reduction of process nodes in ...
9582,2573,Schottky diode integrated into superjunction p...,US11538933,20221227,utility,['H'],['H01'],['H01L'],"['H01L 29/78', 'H01L 21/266', 'H01L 21/28', 'H...",A trench metal-oxide-semiconductor field-effec...,CLAIM OF PRIORITY\nThis application is a conti...,1. A trench metal-oxide-semiconductor field-ef...,H01L,NaN
9583,2574,Methods and apparatus to control zone temperat...,US11538956,20221227,utility,"['H', 'F']","['H01', 'H05', 'F27']","['H01L', 'H05B', 'F27B']","['H01L 31/18', 'H05B 6/06', 'H01L 21/67', 'F27...",Methods and apparatus to control zone temperat...,BACKGROUND\nThis disclosure relates to solar c...,"1. A furnace to fire photovoltaic cells, the f...",H01L,This disclosure relates to solar cell producti...


In [3]:
abstract = df['abstract']
background = df['background']
background = [b for b in background if type(b) == str] # NaN은 제거

In [4]:
len(background)

9293

In [5]:
# 문장 단위로 split

abstract_sent = []
background_sent = []

for a in abstract:
    abstract_sent.extend(sent_tokenize(a))

for b in background:
    background_sent.extend(sent_tokenize(b))

In [6]:
print(len(abstract_sent))
print(len(background_sent))

33512
108915


## 동사 추출

In [7]:
abstract_verbs = []
background_verbs = []

pos_list = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'] # 동사

In [8]:
for sent in tqdm(abstract_sent):
    for vocab in pos_tag(word_tokenize(sent)):
        if vocab[1] in pos_list:
            abstract_verbs.append(vocab[0].lower())

  0%|          | 0/33512 [00:00<?, ?it/s]

In [9]:
for sent in tqdm(background_sent):
    for vocab in pos_tag(word_tokenize(sent)):
        if vocab[1] in pos_list:
            background_verbs.append(vocab[0].lower())

  0%|          | 0/108915 [00:00<?, ?it/s]

In [10]:
abstract_verbs_counter = Counter(abstract_verbs).most_common()
background_verbs_counter = Counter(background_verbs).most_common()
print(len(abstract_verbs_counter))
print(len(background_verbs_counter))

3556
6005


In [11]:
abstract_verbs_counter[:100]

[('is', 17189),
 ('includes', 9765),
 ('forming', 5825),
 ('are', 5314),
 ('formed', 4436),
 ('having', 3462),
 ('provided', 3341),
 ('be', 3091),
 ('configured', 3002),
 ('including', 2742),
 ('etching', 2592),
 ('disposed', 2579),
 ('has', 1988),
 ('form', 1912),
 ('include', 1874),
 ('using', 1655),
 ('wherein', 1383),
 ('removing', 1275),
 ('disclosed', 1064),
 ('being', 1013),
 ('performing', 979),
 ('connected', 958),
 ('comprising', 925),
 ('comprises', 908),
 ('extending', 854),
 ('providing', 835),
 ('holding', 825),
 ('based', 811),
 ('arranged', 741),
 ('coupled', 740),
 ('processing', 729),
 ('depositing', 721),
 ('removed', 704),
 ('supplying', 694),
 ('performed', 683),
 ('manufacturing', 674),
 ('containing', 669),
 ('located', 667),
 ('used', 651),
 ('described', 599),
 ('polishing', 586),
 ('according', 575),
 ('etched', 557),
 ('exposed', 549),
 ('provide', 498),
 ('remove', 491),
 ('provides', 476),
 ('deposited', 470),
 ('positioned', 462),
 ('insulating', 450),
 ('

In [12]:
background_verbs_counter[:100]

[('is', 48747),
 ('be', 21395),
 ('are', 20059),
 ('has', 9510),
 ('used', 8718),
 ('have', 7532),
 ('formed', 7487),
 ('using', 6117),
 ('been', 5875),
 ('etching', 5163),
 ('having', 3544),
 ('forming', 3257),
 ('form', 3046),
 ('relates', 2799),
 ('includes', 2738),
 ('integrated', 2718),
 ('include', 2698),
 ('performed', 2656),
 ('including', 2638),
 ('known', 2113),
 ('provided', 2101),
 ('processing', 2063),
 ('being', 1993),
 ('increased', 1944),
 ('referred', 1845),
 ('provide', 1781),
 ('manufacturing', 1739),
 ('polishing', 1631),
 ('described', 1571),
 ('increasing', 1568),
 ('required', 1560),
 ('removed', 1484),
 ('applied', 1413),
 ('reduce', 1385),
 ('become', 1383),
 ('remove', 1313),
 ('made', 1263),
 ('reduced', 1212),
 ('processes', 1187),
 ('improve', 1147),
 ('removing', 1126),
 ('processed', 1124),
 ('generated', 1100),
 ('supplied', 1092),
 ('disposed', 1083),
 ('deposited', 1082),
 ('increase', 1082),
 ('disclosed', 1076),
 ('discloses', 1074),
 ('cause', 1062)

## Stemming

In [13]:
stemmer = PorterStemmer()

In [14]:
abstract_verbs_stemmed = [stemmer.stem(word) for word in abstract_verbs]
background_verbs_stemmed = [stemmer.stem(word) for word in background_verbs]

In [15]:
abstract_verbs_stemmed_counter = Counter(abstract_verbs_stemmed).most_common()
background_verbs_stemmed_counter = Counter(background_verbs_stemmed).most_common()
print(len(abstract_verbs_stemmed_counter))
print(len(background_verbs_stemmed_counter))

1782
3065


In [16]:
abstract_verbs_stemmed_counter[:100]

[('is', 17189),
 ('includ', 14482),
 ('form', 12310),
 ('are', 5314),
 ('provid', 5150),
 ('be', 4104),
 ('have', 3894),
 ('etch', 3365),
 ('configur', 3017),
 ('dispos', 2675),
 ('remov', 2545),
 ('use', 2370),
 ('compris', 2007),
 ('ha', 1988),
 ('perform', 1923),
 ('extend', 1556),
 ('suppli', 1480),
 ('expos', 1438),
 ('wherein', 1383),
 ('connect', 1277),
 ('deposit', 1259),
 ('process', 1141),
 ('hold', 1121),
 ('disclos', 1118),
 ('contain', 981),
 ('gener', 914),
 ('coupl', 873),
 ('appli', 824),
 ('pattern', 819),
 ('control', 818),
 ('base', 812),
 ('cover', 805),
 ('arrang', 776),
 ('support', 770),
 ('manufactur', 729),
 ('defin', 709),
 ('move', 707),
 ('locat', 678),
 ('receiv', 669),
 ('describ', 659),
 ('polish', 653),
 ('fill', 603),
 ('determin', 599),
 ('posit', 577),
 ('accord', 575),
 ('transfer', 560),
 ('heat', 556),
 ('relat', 542),
 ('measur', 536),
 ('reduc', 532),
 ('place', 513),
 ('fabric', 507),
 ('surround', 485),
 ('separ', 478),
 ('insul', 477),
 ('cont

In [17]:
background_verbs_stemmed_counter[:100]

[('is', 48747),
 ('be', 23388),
 ('are', 20059),
 ('use', 15858),
 ('form', 14011),
 ('have', 11076),
 ('ha', 9510),
 ('includ', 8342),
 ('etch', 7009),
 ('been', 5875),
 ('provid', 5414),
 ('increas', 5026),
 ('process', 4654),
 ('perform', 4403),
 ('remov', 4114),
 ('relat', 3848),
 ('reduc', 3560),
 ('requir', 3092),
 ('integr', 2868),
 ('becom', 2677),
 ('manufactur', 2654),
 ('caus', 2555),
 ('disclos', 2277),
 ('improv', 2246),
 ('produc', 2136),
 ('refer', 2135),
 ('known', 2113),
 ('control', 2103),
 ('appli', 2092),
 ('polish', 2047),
 ('fabric', 2007),
 ('suppli', 1984),
 ('deposit', 1920),
 ('describ', 1903),
 ('gener', 1877),
 ('result', 1691),
 ('need', 1674),
 ('pattern', 1633),
 ('connect', 1513),
 ('scale', 1512),
 ('achiev', 1471),
 ('decreas', 1464),
 ('develop', 1414),
 ('transfer', 1392),
 ('expos', 1359),
 ('allow', 1344),
 ('contain', 1318),
 ('compris', 1295),
 ('creat', 1282),
 ('made', 1263),
 ('clean', 1221),
 ('continu', 1194),
 ('occur', 1129),
 ('dispos', 1

## Lemmatization

In [18]:
lemmatizer = WordNetLemmatizer()

In [19]:
abstract_verbs_lemmatized = [lemmatizer.lemmatize(word, 'v') for word in abstract_verbs]
background_verbs_lemmatized = [lemmatizer.lemmatize(word, 'v') for word in background_verbs]

In [20]:
abstract_verbs_lemmatized_counter = Counter(abstract_verbs_lemmatized).most_common()
background_verbs_lemmatized_counter = Counter(background_verbs_lemmatized).most_common()
print(len(abstract_verbs_lemmatized_counter))
print(len(background_verbs_lemmatized_counter))

1858
3261


In [21]:
abstract_verbs_lemmatized_counter[:100]

[('be', 26773),
 ('include', 14482),
 ('form', 12310),
 ('have', 5882),
 ('provide', 5150),
 ('etch', 3365),
 ('configure', 3017),
 ('dispose', 2675),
 ('remove', 2542),
 ('use', 2370),
 ('comprise', 2007),
 ('perform', 1923),
 ('extend', 1556),
 ('supply', 1480),
 ('expose', 1438),
 ('wherein', 1383),
 ('hold', 1354),
 ('connect', 1277),
 ('deposit', 1257),
 ('process', 1141),
 ('disclose', 1118),
 ('contain', 981),
 ('generate', 913),
 ('couple', 873),
 ('apply', 824),
 ('pattern', 819),
 ('control', 815),
 ('base', 812),
 ('cover', 805),
 ('arrange', 776),
 ('support', 770),
 ('manufacture', 729),
 ('define', 709),
 ('move', 707),
 ('locate', 678),
 ('receive', 669),
 ('describe', 659),
 ('polish', 653),
 ('fill', 603),
 ('determine', 599),
 ('position', 577),
 ('accord', 575),
 ('transfer', 560),
 ('heat', 556),
 ('relate', 542),
 ('measure', 536),
 ('reduce', 532),
 ('place', 513),
 ('fabricate', 506),
 ('surround', 485),
 ('separate', 478),
 ('insulate', 477),
 ('contact', 468),


In [22]:
background_verbs_lemmatized_counter[:100]

[('be', 98757),
 ('have', 20702),
 ('use', 15858),
 ('form', 14011),
 ('include', 8342),
 ('etch', 7009),
 ('provide', 5414),
 ('increase', 5026),
 ('process', 4654),
 ('perform', 4402),
 ('remove', 4101),
 ('relate', 3848),
 ('reduce', 3560),
 ('require', 3092),
 ('integrate', 2867),
 ('become', 2702),
 ('manufacture', 2654),
 ('cause', 2555),
 ('make', 2383),
 ('disclose', 2277),
 ('improve', 2246),
 ('produce', 2136),
 ('know', 2134),
 ('refer', 2132),
 ('control', 2098),
 ('apply', 2092),
 ('polish', 2047),
 ('supply', 1984),
 ('fabricate', 1937),
 ('deposit', 1919),
 ('describe', 1903),
 ('generate', 1877),
 ('result', 1691),
 ('need', 1674),
 ('pattern', 1633),
 ('hold', 1576),
 ('connect', 1513),
 ('scale', 1512),
 ('achieve', 1471),
 ('decrease', 1464),
 ('develop', 1412),
 ('transfer', 1392),
 ('expose', 1359),
 ('allow', 1344),
 ('contain', 1315),
 ('comprise', 1295),
 ('create', 1282),
 ('show', 1243),
 ('clean', 1219),
 ('continue', 1187),
 ('occur', 1129),
 ('dispose', 112

## Save

In [83]:
with open('result/verb_abstract_stemmed.pickle','wb') as fw:
    pickle.dump(abstract_verbs_stemmed_counter, fw)
with open('result/verb_background_stemmed.pickle','wb') as fw:
    pickle.dump(background_verbs_stemmed_counter, fw)

with open('result/verb_abstract_lemmatized.pickle','wb') as fw:
    pickle.dump(abstract_verbs_lemmatized_counter, fw)
with open('result/verb_background_lemmatized.pickle','wb') as fw:
    pickle.dump(background_verbs_lemmatized_counter, fw)

## Load

In [1]:
import pickle

with open('result/verb_abstract_stemmed.pickle', 'rb') as fr:
    abstract_verbs_stemmed_counter = pickle.load(fr)
with open('result/verb_background_stemmed.pickle', 'rb') as fr:
    background_verbs_stemmed_counter = pickle.load(fr)

with open('result/verb_abstract_lemmatized.pickle', 'rb') as fr:
    abstract_verbs_lemmatized_counter = pickle.load(fr)
with open('result/verb_background_lemmatized.pickle', 'rb') as fr:
    background_verbs_lemmatized_counter = pickle.load(fr)

## Technical Entity 2개 이상 포함된 문장만
- 한 문장 내에서 technical entity가 2개 이상 포함됐을 때만 동사 추출
- Abstract만

In [23]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.SIMPLE,
        )
        keyphrases = [result.get("word").strip() for result in results] # 같은 keyphrase도 중복해서 추출

        return results, keyphrases

In [24]:
# CUDA 연결 안 됨, GPU 사용 x
# Load pipeline
model_name = "ml6team/keyphrase-extraction-kbir-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

Using /home/hjkim/.cache/torch_extensions/py39_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Failed to load CUDA kernels. Mra requires custom CUDA kernels. Please verify that compatible versions of PyTorch and CUDA Toolkit are installed: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.


In [31]:
abstract_verbs = []
pos_list = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'] # 동사

for sent in tqdm(abstract_sent):
    raw, words = extractor(sent)
    if len(words) >= 2:
        for vocab in pos_tag(word_tokenize(sent)):
            if vocab[1] in pos_list:
                abstract_verbs.append(vocab[0].lower())

  0%|          | 0/33512 [00:00<?, ?it/s]

In [32]:
abstract_verbs_counter = Counter(abstract_verbs).most_common()
print(len(abstract_verbs_counter))

3319


In [33]:
abstract_verbs_counter

[('is', 12518),
 ('includes', 8285),
 ('forming', 4970),
 ('formed', 3537),
 ('are', 3533),
 ('having', 3080),
 ('configured', 2656),
 ('including', 2514),
 ('be', 2302),
 ('etching', 2279),
 ('provided', 2199),
 ('disposed', 2190),
 ('form', 1696),
 ('has', 1502),
 ('using', 1436),
 ('include', 1388),
 ('wherein', 1319),
 ('removing', 1092),
 ('being', 910),
 ('performing', 874),
 ('comprising', 843),
 ('connected', 804),
 ('holding', 779),
 ('comprises', 774),
 ('extending', 765),
 ('providing', 735),
 ('based', 706),
 ('coupled', 637),
 ('depositing', 637),
 ('supplying', 632),
 ('containing', 610),
 ('arranged', 604),
 ('located', 568),
 ('disclosed', 560),
 ('processing', 552),
 ('performed', 541),
 ('manufacturing', 517),
 ('polishing', 511),
 ('used', 509),
 ('according', 507),
 ('exposed', 465),
 ('removed', 452),
 ('insulating', 444),
 ('etched', 428),
 ('applying', 419),
 ('remove', 414),
 ('provide', 413),
 ('positioned', 398),
 ('exposing', 397),
 ('patterning', 362),
 ('de

In [34]:
# Lemmatization
lemmatizer = WordNetLemmatizer()

abstract_verbs_lemmatized = [lemmatizer.lemmatize(word, 'v') for word in abstract_verbs]

abstract_verbs_lemmatized_counter = Counter(abstract_verbs_lemmatized).most_common()
print(len(abstract_verbs_lemmatized_counter))

1730


In [35]:
abstract_verbs_lemmatized_counter[:100]

[('be', 19390),
 ('include', 12266),
 ('form', 10312),
 ('have', 4903),
 ('provide', 3678),
 ('etch', 2888),
 ('configure', 2670),
 ('dispose', 2281),
 ('remove', 2012),
 ('use', 1999),
 ('comprise', 1746),
 ('perform', 1639),
 ('supply', 1331),
 ('wherein', 1319),
 ('extend', 1290),
 ('expose', 1253),
 ('hold', 1248),
 ('connect', 1088),
 ('deposit', 1040),
 ('contain', 870),
 ('process', 869),
 ('generate', 810),
 ('couple', 759),
 ('apply', 734),
 ('cover', 721),
 ('base', 707),
 ('control', 683),
 ('pattern', 683),
 ('support', 652),
 ('arrange', 635),
 ('define', 604),
 ('disclose', 600),
 ('move', 600),
 ('locate', 578),
 ('receive', 569),
 ('polish', 562),
 ('manufacture', 558),
 ('accord', 507),
 ('position', 501),
 ('determine', 497),
 ('heat', 487),
 ('fill', 484),
 ('transfer', 480),
 ('insulate', 467),
 ('measure', 459),
 ('mount', 432),
 ('place', 431),
 ('reduce', 431),
 ('surround', 417),
 ('separate', 416),
 ('contact', 390),
 ('detect', 379),
 ('relate', 377),
 ('bond'

In [36]:
# Save
with open('result/verb_abstract_lemmatized_reduced.pickle','wb') as fw:
    pickle.dump(abstract_verbs_lemmatized_counter, fw)

In [1]:
# Load
import pickle

with open('result/verb_abstract_lemmatized_reduced.pickle', 'rb') as fr:
    abstract_verbs_lemmatized_counter = pickle.load(fr)

In [2]:
abstract_verbs_lemmatized_counter[:100]

[('be', 19390),
 ('include', 12266),
 ('form', 10312),
 ('have', 4903),
 ('provide', 3678),
 ('etch', 2888),
 ('configure', 2670),
 ('dispose', 2281),
 ('remove', 2012),
 ('use', 1999),
 ('comprise', 1746),
 ('perform', 1639),
 ('supply', 1331),
 ('wherein', 1319),
 ('extend', 1290),
 ('expose', 1253),
 ('hold', 1248),
 ('connect', 1088),
 ('deposit', 1040),
 ('contain', 870),
 ('process', 869),
 ('generate', 810),
 ('couple', 759),
 ('apply', 734),
 ('cover', 721),
 ('base', 707),
 ('control', 683),
 ('pattern', 683),
 ('support', 652),
 ('arrange', 635),
 ('define', 604),
 ('disclose', 600),
 ('move', 600),
 ('locate', 578),
 ('receive', 569),
 ('polish', 562),
 ('manufacture', 558),
 ('accord', 507),
 ('position', 501),
 ('determine', 497),
 ('heat', 487),
 ('fill', 484),
 ('transfer', 480),
 ('insulate', 467),
 ('measure', 459),
 ('mount', 432),
 ('place', 431),
 ('reduce', 431),
 ('surround', 417),
 ('separate', 416),
 ('contact', 390),
 ('detect', 379),
 ('relate', 377),
 ('bond'